### Installation des bibliothèques 

pip install numpy  
pip install pandas 
pip install scikit-learn
pip install torch
pip install transformers
pip install datasets

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import transformers

2024-08-13 10:08:11.356910: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-13 10:08:11.373179: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 10:08:11.392944: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 10:08:11.398658: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-13 10:08:11.413466: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from transformers import TFBertModel, BertConfig, BertTokenizer, BertTokenizerFast, BertModel
from transformers import AutoTokenizer
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from datasets import Dataset

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np

## Data


In [3]:
file1='Data clear/fichier_combiné.csv'
df = pd.read_csv(file1)
print(df.shape,df.head())

(33471, 2)                                                Block  Label
0  ﻿The Project Gutenberg eBook of Contes Françai...      3
1  You may copy it, give it away or re-use it und...      3
2  If you are not located in the United States, y...      3
3  Title: Contes Français Editor: Douglas Labaree...      3
4                                                 D.      3


In [4]:
# Mélanager le dataframe
df_suffle = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(df_suffle.shape)
print(df_suffle.head())

(33471, 2)
                                               Block  Label
0  Un cordon de grosses pierres fixait tout autou...      1
1  Vous dirai-je pourtant que j'ai perdu quelques...      2
2  lèvre, _f._ lip; lèvres en fleur, full-blown l...      3
3  Bonaparte (Napoléon), the French Emperor (1769...      3
4  mengheldichten: fyghes noeper;  Bacchus-Cortryck.      2


In [5]:
# Convertir les label du types int en type float 
df_suffle['Label']=df_suffle['Label'].astype(float)

### 1. Encode labels


In [6]:
one=OneHotEncoder(sparse_output=False)
encoded=one.fit_transform(df_suffle[['Label']])
len(encoded)

33471

### 2. Creat new data with the encoded value

In [7]:
head = {'text' : [], 'label' : []}
df_model = pd.DataFrame(head,dtype=object)

In [9]:
for k in tqdm(range (len(df))):
    df_model.loc[k]=[df_suffle['Block'][k],encoded[k]]
df_model.head(10)

100%|██████████| 33471/33471 [00:01<00:00, 19899.68it/s]


,text,label
0,Un cordon de grosses pierres fixait tout autou...,"[0.0, 1.0, 0.0, 0.0]"
1,Vous dirai-je pourtant que j'ai perdu quelques...,"[0.0, 0.0, 1.0, 0.0]"
2,"lèvre, _f._ lip; lèvres en fleur, full-blown l...","[0.0, 0.0, 0.0, 1.0]"
3,"Bonaparte (Napoléon), the French Emperor (1769...","[0.0, 0.0, 0.0, 1.0]"
4,mengheldichten: fyghes noeper; Bacchus-Cortryck.,"[0.0, 0.0, 1.0, 0.0]"
5,Vous aviez donc des livres?,"[1.0, 0.0, 0.0, 0.0]"
6,Ah çà!,"[0.0, 1.0, 0.0, 0.0]"
7,"faire, _v._ to do, make, form, build, perform,...","[0.0, 0.0, 0.0, 1.0]"
8,Sa présence me rendit ma fermeté.,"[0.0, 0.0, 0.0, 1.0]"
9,"grouiller, _v._ to swarm, crawl.","[0.0, 0.0, 0.0, 1.0]"


In [11]:
print(type(df_model['text'][2]))
print(type(df_model['label'][0]))

<class 'str'>
<class 'numpy.ndarray'>


### 3. Split the data into train and test

In [12]:
#Split data
train_df, test_df = train_test_split(df_model,test_size=0.2)
print(len(train_df),len(test_df))

26776 6695


In [13]:
# Check the max len 
ex_len = []
for i in train_df['text']:
  word_count = len(str(i).split())
  ex_len.append(word_count)
print('max length of text: ', max(ex_len), 'words')

max length of text:  760 words


In [14]:
print(train_df.head())
print('--------------------------------')
print(test_df.head())

                                                    text                 label
13504  A six heures, bélître, et nous nous battrons à...  [0.0, 0.0, 1.0, 0.0]
13253  sauter, _v._ to jump, leap, skip, leap over, p...  [0.0, 0.0, 0.0, 1.0]
12937  --C'est votre état de dire cela, monsieur l'ab...  [1.0, 0.0, 0.0, 0.0]
9750   Çà, comment comprenez-vous le but élevé que j'...  [0.0, 0.0, 1.0, 0.0]
30099                                              (rel.  [0.0, 0.0, 0.0, 1.0]
--------------------------------
                                                    text                 label
27317  «Ainsi, continua l'Anglais, qui reprit le prem...  [1.0, 0.0, 0.0, 0.0]
4522   Un grand nombre de vieux livres, étalés sous c...  [0.0, 0.0, 1.0, 0.0]
13888                                               à...  [0.0, 1.0, 0.0, 0.0]
7354         dit Gueulemer, les coqueurs peuvent passer.  [0.0, 1.0, 0.0, 0.0]
32604        talonner, _v._ to be close on the heels of.  [0.0, 0.0, 0.0, 1.0]


In [15]:
type(train_df)

pandas.core.frame.DataFrame

In [16]:
# Convertir les DataFrames en objets Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

### Tokenize
Tranfromer les blocks de texte en vecteur pour être donné à Bert

In [17]:
# Définir le modèle et charger le tokenizer
bert_model = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(bert_model)

In [18]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

# Appliquer la tokenisation aux datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 6695/6695 [00:00<00:00, 10544.57 examples/s]


### Préparer les étiquettes 

In [19]:
def convert_labels_to_indices(labels):
    return [np.argmax(label) for label in labels]

# Convertir les étiquettes one-hot en indices
train_dataset = train_dataset.map(lambda examples: {'labels': convert_labels_to_indices(examples['label'])}, batched=True)
test_dataset = test_dataset.map(lambda examples: {'labels': convert_labels_to_indices(examples['label'])}, batched=True)

# Réorganiser les colonnes pour être compatibles avec le modèle
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map: 100%|██████████| 6695/6695 [00:00<00:00, 56003.58 examples/s]


### Model 

In [23]:
# Charger le modèle BERT pré-entraîné et ajouter une couche de classification
num_labels = len(set(convert_labels_to_indices(df['Label'])))  # Assurez-vous que num_labels est correct
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',          # Répertoire pour sauvegarder les résultats
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_dir='./logs',            # Répertoire pour les logs
    logging_steps=10,                # Fréquence des logs
)

# Créer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Entraîner le modèle
trainer.train()

# Évaluer le modèle
results = trainer.evaluate()
print(results)


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

RuntimeError: Found dtype Long but expected Float

### Configuration de Bert

In [ ]:
model_config = BertConfig.from_pretrained(bert_model)

In [ ]:
model_config.output_hidden_states = True
model_config

### Parameters

In [ ]:
bert = TFBertModel.from_pretrained(bert_model, config = model_config)


In [ ]:
# tunable parameters
max_len = 512
#fix learning_rate
learning_rate = 1e-4
epochs = 5

optimizer = Adam(learning_rate = learning_rate)
metrics=tf.keras.metrics.BinaryAccuracy(
    name="binary_accuracy", dtype=None, threshold=0.5
)
loss = 'binary_crossentropy'


In [ ]:
# Exemple de longueur maximale pour les séquences
max_len = 512

# Définir les entrées
input_ids_i = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
token_type_ids_i = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='token_type_ids')
attention_mask_i = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')

# Obtenir les sorties du modèle BERT
bert_output = bert([input_ids_i, token_type_ids_i, attention_mask_i])[0]


In [ ]:
# Définition des entrées
input_ids_i = Input(shape = (max_len, ), dtype = tf.int32, name = 'input_ids')
token_type_ids_i = Input(shape = (max_len, ), dtype = tf.int32, name = 'token_type_ids')
attention_mask_i = Input(shape = (max_len, ), dtype = tf.int32, name = 'attention_mask')
inputs = [input_ids_i, token_type_ids_i, attention_mask_i]

bert_output = bert(input_ids_i, token_type_ids = token_type_ids_i, attention_mask = attention_mask_i)[0]
